Import pyspark

In [1]:
#import spark
from pyspark.sql import SparkSession
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 ass2 BNPL group 28")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/09/02 14:26:17 WARN Utils: Your hostname, Rudyletsgo resolves to a loopback address: 127.0.1.1; using 172.23.255.243 instead (on interface eth0)
22/09/02 14:26:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/09/02 14:26:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [35]:
# Read the merchants and consumer parquet data
merchants = spark.read.parquet('../data/tables/tbl_merchants.parquet')
consumers = spark.read.parquet('../data/tables/consumer_user_details.parquet')
transactions = spark.read.parquet('../data/tables/transactions_20210228_20210827_snapshot')
consumers_csv = spark.read.options(header='True', inferSchema='True', delimiter='|') \
  .csv('../data/tables/tbl_consumer.csv')


In [3]:
# The merchants data size
print(merchants.count(),len(merchants.columns))

4026 3


In [4]:
merchants

name,tags,merchant_abn
Felis Limited,"((furniture, home...",10023283211
Arcu Ac Orci Corp...,"([cable, satellit...",10142254217
Nunc Sed Company,"([jewelry, watch,...",10165489824
Ultricies Digniss...,"([wAtch, clock, a...",10187291046
Enim Condimentum PC,([music shops - m...,10192359162
Fusce Company,"[(gift, card, nov...",10206519221
Aliquam Enim Inco...,"[(computers, comP...",10255988167
Ipsum Primis Ltd,"[[watch, clock, a...",10264435225
Pede Ultrices Ind...,([computer progra...,10279061213
Nunc Inc.,"[(furniture, home...",10323485998


In [5]:
# The mechants data column types
merchants.dtypes

[('name', 'string'), ('tags', 'string'), ('merchant_abn', 'bigint')]

In [6]:
consumers

user_id,consumer_id
1,1195503
2,179208
3,1194530
4,154128
5,712975
6,407340
7,511685
8,448088
9,650435
10,1058499


In [41]:
consumers_csv.groupBy('state','postcode').count().show()

+-----+--------+-----+
|state|postcode|count|
+-----+--------+-----+
|  VIC|    3279|  175|
|   SA|    5261|  154|
|  NSW|    2334|  181|
|  NSW|    1440|  150|
|  NSW|    1430|  149|
|   SA|    5540|  163|
|  NSW|    2397|  155|
|  VIC|    3194|  159|
|  VIC|    3490|  155|
|   SA|    5630|  168|
|  NSW|    2190|  172|
|  QLD|    4133|  141|
|  ACT|    2620|  143|
|   WA|    6405|  174|
|  NSW|    2024|  160|
|  VIC|    3239|  162|
|  NSW|    1119|  164|
|  VIC|    3759|  164|
|  QLD|    4885|  160|
|   SA|    5266|  161|
+-----+--------+-----+
only showing top 20 rows



In [42]:
consumers_csv

name,address,state,postcode,gender,consumer_id
Yolanda Williams,413 Haney Gardens...,WA,6935,Female,1195503
Mary Smith,3764 Amber Oval,NSW,2782,Female,179208
Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530
Lindsay Jimenez,00653 Davenport C...,NSW,2780,Female,154128
Rebecca Blanchard,9271 Michael Mano...,WA,6355,Female,712975
Karen Chapman,2706 Stewart Oval...,NSW,2033,Female,407340
Andrea Jones,122 Brandon Cliff,QLD,4606,Female,511685
Stephen Williams,6804 Wright Crest...,WA,6056,Male,448088
Stephanie Reyes,5813 Denise Land ...,NSW,2482,Female,650435
Jillian Gonzales,461 Ryan Common S...,VIC,3220,Female,1058499


In [8]:
transactions

user_id,merchant_abn,dollar_value,order_id,order_datetime
18478,62191208634,63.255848959735246,949a63c8-29f7-4ab...,2021-08-20
2,15549624934,130.3505283105634,6a84c3cf-612a-457...,2021-08-20
18479,64403598239,120.15860593212783,b10dcc33-e53f-425...,2021-08-20
3,60956456424,136.6785200286976,0f09c5a5-784e-447...,2021-08-20
18479,94493496784,72.96316578355305,f6c78c1a-4600-4c5...,2021-08-20
3,76819856970,448.529684285612,5ace6a24-cdf0-4aa...,2021-08-20
18479,67609108741,86.4040605836911,d0e180f0-cb06-42a...,2021-08-20
3,34096466752,301.5793450525113,6fb1ff48-24bb-4f9...,2021-08-20
18482,70501974849,68.75486276223054,8505fb33-b69a-412...,2021-08-20
4,49891706470,48.89796461900801,ed11e477-b09f-4ae...,2021-08-20


In [9]:
# get data sizes of transactions and consumers
print("The transactions size is")
print(transactions.count(),len(transactions.columns))

print("The consumers size is")
print(consumers.count(),len(consumers.columns))

The transactions size is
3643266 5
The consumers size is
499999 2
